In [1]:
import pandas as pd
import numpy as np
import datetime
import pyxirr
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.fundamentaldata import FundamentalData
# from alpha_vantage.foreignexchange import ForeignExchange
# from alpha_vantage.fundamentaldata import FundamentalData
from secrets import ROOT, ALPHAVANTAGE_API_KEY, ACCOUNT_CURRENCY

In [2]:
class av:
    try:
        ts = TimeSeries(key=ALPHAVANTAGE_API_KEY, output_format='pandas', indexing_type='date')
        cc = ForeignExchange(key=ALPHAVANTAGE_API_KEY, output_format='pandas', indexing_type='date')
        fd = FundamentalData(key=ALPHAVANTAGE_API_KEY, output_format='pandas')

    except ValueError:
        print('Please provide a valid AlphaVantage API key.\nGet a free key from https://www.alphavantage.co/support/#api-key')

In [3]:
# HELPERS
def series(value, start, end=datetime.date.today()):
    return pd.Series(value ,index=pd.date_range(start=start, end=end, freq='B'))

def conc_sum(df1, df2, intersect=False):
    if intersect:
        index = df1.index.intersection(df2.index)
        df1 = df1.loc[index]
        df2 = df2.loc[index]
    return pd.concat([df1, df2], axis=1).sum(1)

def conc_mul(df1, df2, intersect=False):
    if intersect:
        index = df1.index.intersection(df2.index)
        df1 = df1.loc[index]
        df2 = df2.loc[index]
    return pd.concat([df1, df2], axis=1).mul(1)

a = series(1,'2020-01-01','2020-01-20')
b = series(2,'2020-01-10','2020-01-20')
c = series(-1,'2020-01-15','2020-01-20')
conc_sum(c,a, True)

2020-01-15    0
2020-01-16    0
2020-01-17    0
2020-01-20    0
Freq: B, dtype: int64

In [4]:
class Account:

    def __init__(self, currency):
        self.currency = currency
        self.name = 'CASH.' + self.currency
        self.internal_transactions = pd.Series()
        self.external_transactions = pd.Series()
    
    def internal_flow(self, date, quantity):
        self.internal_transactions = self.internal_transactions.append(
            pd.Series(
                quantity,
                index=[date])).sort_index()

    def external_flow(self, date, quantity):
        self.external_transactions = self.external_transactions.append(
            pd.Series(
                quantity,
                index=[date])).sort_index()

    def run(self):
        self.holdings = self.internal_transactions.cumsum()
        self.invested_capital = self.external_transactions.cumsum()

class Security:

    def __init__(self, name):
        self.name = name
        self.transactions = pd.Series()
        self.data = av.ts.get_daily_adjusted(
            self.name,
            outputsize='full')[0].iloc[::-1]
        self.data = self.data.asfreq('B', method='ffill')
        self.prices = self.data['4. close']
        self.dividends = self.data['7. dividend amount']
        self.splits = self.data['8. split coefficient']

    def record(self, date, quantity):
        self.transactions = self.transactions.append(
            pd.Series(
                quantity,
                index=[date])).sort_index()
        '''
        self.holdings = self.transactions.reindex(
            pd.date_range(
                start=self.transactions.index[0],
                end=datetime.date.today(),
                freq='B'),
            fill_value=0).cumsum() * self.splits.loc[self.transactions.index[0]:].cumprod()
        self.holdings_value = self.holdings * self.prices.loc[self.transactions.index[0]:]
        self.holdings_dividend = self.holdings * self.dividends.loc[self.transactions.index[0]:]
        '''

    def run(self):
        self.holdings = self.transactions.cumsum() * self.splits
        self.holdings_value_locale = self.holdings * self.prices
        self.holdings_value = self.holdings_value_locale # / fx_rate
        self.holdings_dividend_locale = self.holdings * self.dividends
        self.holdings_dividend = self.holdings_dividend_locale # / fx_rate


In [5]:
class Portfolio:
    
    def __init__(self, currency='USD'):
        self.data = pd.DataFrame()
        self.currency = currency
        self.account = Account(self.currency)
        self.securities = dict() # dictionary with Security object at key='TICKER NAME'
        self.start_date = datetime.date.today()

    def add_transaction(self, transaction):
        if transaction.Date < self.start_date:
            self.start_date = transaction.Date
        getattr(self, transaction.Order)(transaction)

    def deposit(self, transaction):
        self.account.internal_flow(transaction.Date, transaction.Quantity * transaction.Price - transaction.Fee)
        self.account.external_flow(transaction.Date, transaction.Quantity * transaction.Price)

    def withdrawal(self, transaction):
        self.account.internal_flow(transaction.Date, - transaction.Quantity * transaction.Price - transaction.Fee)
        self.account.external_flow(transaction.Date, - transaction.Quantity * transaction.Price)

    def purchase(self, transaction):
        self.account.internal_flow(transaction.Date, - transaction.Quantity * transaction.Price - transaction.Fee)
        tick = self.securities.setdefault(transaction.Ticker, Security(transaction.Ticker))
        tick.record(transaction.Date, transaction.Quantity)

    def sale(self, transaction):
        self.account.internal_flow(transaction.Date, transaction.Quantity * transaction.Price - transaction.Fee)
        tick = self.securities.setdefault(transaction.Ticker, Security(transaction.Ticker))
        tick.record(transaction.Date, - transaction.Quantity)

    def join_holdings(self):
        self.holdings = pd.DataFrame()
        '''
        for security in self.securities.values():
            security.run()
            self.account.internal_transactions = self.account.internal_transactions + security.holdings_dividend
            self.holdings = self.holdings.join(security.holdings_value.rename(security.name))
        self.account.run()
        self.holdings = self.holdings.join(self.account.holdings.rename(self.account.name))
        '''

    def run(self):
        for date, transaction in self.data.iterrows():
            self.add_transaction(date, transaction)
        # self.join_holdings()
        # self.generate_stats()


In [6]:
aapl = Security('AAPL')
aapl.record(pd.to_datetime('2021-09-30'), -100)
aapl.record(pd.to_datetime('2021-09-21'), 300)
aapl.record(pd.to_datetime('2021-09-10'), 200)
aapl.transactions

/var/folders/bk/yh3kcqrn6c1gx1yqf1lbzb1w0000gn/T/ipykernel_2073/1935690679.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.transactions = pd.Series()


2021-09-10    200
2021-09-21    300
2021-09-30   -100
dtype: int64

In [7]:
data=pd.read_csv(ROOT+"/assets/portfolios/generic.csv",sep=",",parse_dates=True)

In [9]:
portfolio=Portfolio()

/var/folders/bk/yh3kcqrn6c1gx1yqf1lbzb1w0000gn/T/ipykernel_2073/1935690679.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.internal_transactions = pd.Series()
/var/folders/bk/yh3kcqrn6c1gx1yqf1lbzb1w0000gn/T/ipykernel_2073/1935690679.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.external_transactions = pd.Series()


In [10]:
for i, transaction in data.iterrows():
    portfolio.add(transaction)

/var/folders/bk/yh3kcqrn6c1gx1yqf1lbzb1w0000gn/T/ipykernel_2073/1935690679.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.transactions = pd.Series()


In [11]:
portfolio.account.internal_transactions

2019-10-01    100000.00
2019-10-12    -20672.76
2019-11-24    -37100.00
2019-12-04     13368.00
2020-01-06    -20810.00
2020-01-25    -30000.00
dtype: float64